## Tasks Initiated This Trimester

### Gamify

While developing the game engine, it was my job to make a leaderboard. 

- <b>Iteration 1:</b> The leaderboard initially started off as a solely input-based feature that was yet to be connected to the save score feature. Pretty much a prototype, couldn't be used in real-time yet.

- <b>Iteration 2:</b> BE API and data table were established, the leaderboard now pulled the name and the scores from the "save_score" table in the backend. The two features were integrated, leaderboard worked in real-time.

- <b>Iteration 3:</b> Division of leaderboard into two distinct types: DYNAMIC & ELEMENTARY! Dynamic would be the user playing a game, saving their score, and their entry automatically showing up on the leaderboard if it's in the top 10 scores. Elementary was targetted for CSSE students for them to deepen their understanding of CRUD operations and see their inputs come to life. Input-style leaderboard from iteration 1 was brought back. User now got an option to pick between either for their game screen.

- <b>Iteration 4:</b> Specific table in the backend for leaderboard was deleted and instead I used the common table created by my peer. Functionality for toggling leaderboard was added. The correct name is now displayed on dynamic as it pulls from the authorized user's credentials.

<img src="{{site.baseurl}}/images/ui_be_flow.png"/>
<img src="{{site.baseurl}}/images/leaderboard_mermaid.png"/>

### Adding a Score through Elementary Option

{% highlight python %}
        async addElementaryScore() {
        console.log('=== ADD ELEMENTARY SCORE ===');
        const nameInput = document.getElementById('player-name');
        const scoreInput = document.getElementById('player-score');

        const name = nameInput.value.trim();
        const score = parseInt(scoreInput.value);

        if (!name || isNaN(score)) {
            alert('Please enter both name and score');
            return;
        }

        const endpoint = '/api/events/ELEMENTARY_LEADERBOARD';
        console.log('POST endpoint:', endpoint);

        // If backend is unavailable, save locally in localStorage and update UI
        if (!this.hasBackend && !window.javaBackendUrl) {
            const storageKey = `elementary_leaderboard_${this.gameName}`;
            const stored = JSON.parse(localStorage.getItem(storageKey) || '[]');
            const entry = {
                id: `local-${Date.now()}`,
                payload: { user: name, score: score, gameName: this.gameName },
                timestamp: new Date().toISOString()
            };
            stored.push(entry);
            localStorage.setItem(storageKey, JSON.stringify(stored));

            // Clear inputs and refresh local display
            nameInput.value = '';
            scoreInput.value = '';
            await this.fetchElementaryLeaderboard();
            return;
        }

        try {
            const base =
                window.javaBackendUrl ||
                (location.hostname === 'localhost' ? 'http://localhost:8585' : javaURI);

            const url = `${base.replace(/\/$/, '')}${endpoint}`;
            console.log('Full URL:', url);

            // Create payload matching Java backend AlgorithmicEvent structure
            const requestBody = {
                payload: {
                    user: name,
                    score: score,
                    gameName: this.gameName
                }
            };
            console.log('Payload:', JSON.stringify(requestBody));

            // POST to backend - using fetchOptions for proper authentication
            const res = await fetch(
                url,
                {
                    ...fetchOptions,
                    method: 'POST',
                    headers: {
                        ...fetchOptions?.headers,
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify(requestBody)
                }
            );
{% endhighlight %}


### Deleting a Score through Elementary Leaderboard

{% highlight python %}
        async deleteElementaryScore(id) {
        if (!confirm('Are you sure you want to delete this score?')) {
            return;
        }

        console.log('=== DELETE SCORE ===');
        console.log('Deleting ID:', id);

        try {
            // If backend unavailable, delete from localStorage
            if (!this.hasBackend && !window.javaBackendUrl) {
                const storageKey = `elementary_leaderboard_${this.gameName}`;
                const stored = JSON.parse(localStorage.getItem(storageKey) || '[]');
                const filtered = stored.filter(e => e.id !== id);
                localStorage.setItem(storageKey, JSON.stringify(filtered));
                await this.fetchElementaryLeaderboard();
                return;
            }

            const base =
                window.javaBackendUrl ||
                (location.hostname === 'localhost' ? 'http://localhost:8585' : javaURI);

            const url = `${base.replace(/\/$/, '')}/api/events/ELEMENTARY_LEADERBOARD/${id}`;
            console.log('DELETE URL:', url);

            // DELETE from backend - using fetchOptions for proper authentication
            const res = await fetch(
                url,
                {
                    ...fetchOptions,
                    method: 'DELETE'
                }
            );

            if (!res.ok) {
                const errorText = await res.text();
                console.error('Delete failed:', res.status, errorText);
                throw new Error(`Failed to delete: ${res.status} - ${errorText}`);
            }

            console.log('Score deleted successfully');

            // Fetch updated leaderboard from backend
            await this.fetchElementaryLeaderboard();

        } catch (error) {
            console.error('Error deleting score:', error);
            alert(`Failed to delete score: ${error.message}`);
        }
    }
{% endhighlight %}

### Fetching User and Score through Dynamic Leaderboard

{% highlight python %}
        async fetchLeaderboard() {
        if (this.mode !== 'dynamic') return;

        const list = document.getElementById('leaderboard-list');
        if (!list) return;
        try {
            // If backend unavailable, load local scores
            if (!this.hasBackend && !window.javaBackendUrl) {
                const storageKey = `score_counter_${this.gameName}`;
                const stored = JSON.parse(localStorage.getItem(storageKey) || '[]');
                const transformed = stored.map(e => ({
                    id: e.id,
                    payload: { user: e.payload.user, score: e.payload.score, gameName: e.payload.gameName },
                    timestamp: e.timestamp
                }));
                this.displayLeaderboard(transformed);
                return;
            }

            const base =
                window.javaBackendUrl ||
                (location.hostname === 'localhost' ? 'http://localhost:8585' : javaURI);

            const res = await fetch(
                `${base.replace(/\/$/, '')}/api/events/SCORE_COUNTER`,
                { 
                    ...fetchOptions, 
                    method: 'GET'
                }
            );

            if (!res.ok) throw new Error(`HTTP ${res.status}: ${res.statusText}`);
            const data = await res.json();
            this.displayLeaderboard(data);
        } catch (err) {
            console.error('Error fetching dynamic leaderboard:', err);
            // Check for authentication errors (401 or 403 status)
            if (err.message && (err.message.includes('401') || err.message.includes('403'))) {
                list.innerHTML = `<p class="error">Please login to access this feature.</p>`;
            } else {
                list.innerHTML = `<p class="error">Failed to load leaderboard</p>`;
            }
        }
    }
{% endhighlight %}

Implemented "e" Interact Feature for the Chicken and the End Portal

<video controls>
  <source src="{{site.baseurl}}/images/end_dimension.mov">
  Your browser does not support the video tag.
</video>

<video controls>
  <source src="{{site.baseurl}}/images/plains.mov">
  Your browser does not support the video tag.
</video>

### Going Forward Into Trimester 3

[Team Methology & Tasks for Future](https://github.com/SoniDhenuva/HawkHub/issues/1)

## Wednesday Prep for AP Exam

### Our Lesson

<a href="https://pages.opencodingsociety.com/csa/frqs/2015/4"> <button style="background-color: #F88379; color: white; padding: 10px"> 2015 - FRQ 4 </button> </a>

### HW from Other's Lessons

<a href="https://github.com/avikaprasad22/avika_2025/issues/23"> <button style="background-color: #F88379; color: white; padding: 10px"> 2013 FRQ 3 </button> </a>
<a href="https://github.com/avikaprasad22/avika_2025/issues/22"> <button style="background-color: #F88379; color: white; padding: 10px"> 2024 FRQ 1 </button> </a>
<a href="https://github.com/avikaprasad22/avika_2025/issues/21"> <button style="background-color: #F88379; color: white; padding: 10px"> 2024 FRQ 2 </button> </a>